In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessing as pu
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,\
GradientBoostingClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier\
,AdaBoostClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from collections import Counter

In [10]:
bst_col=pickle.load(open('bst_col.pickle','rb'))

In [11]:
df=pd.read_csv('income_evaluation.csv')
pd.set_option('display.max_columns', None)
df=pu.SpaceRemove(df)
y=df['income'].apply(lambda x:0 if x==' <=50K' else 1)
X=df.drop(['income'],axis=1)
X,Encoders=pu.Cat2Num(X)
X=X[bst_col]

['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']


In [12]:
X.head()

,age,capital_gain,marital_status_ Married-civ-spouse,final_weight,relationship_ Husband,education_num,hrs_per_week,education,occupation,marital_status_ Never-married,workclass,sex_ Female,relationship_ Not-in-family,relationship_ Own-child,relationship_ Wife,native_country,race_ White,marital_status_ Divorced,race_ Black
0,39,2174,0.0,77516,0.0,13,40,9,1,1.0,7,0.0,1.0,0.0,0.0,39,1.0,0.0,0.0
1,50,0,1.0,83311,1.0,13,13,9,4,0.0,6,0.0,0.0,0.0,0.0,39,1.0,0.0,0.0
2,38,0,0.0,215646,0.0,9,40,11,6,0.0,4,0.0,1.0,0.0,0.0,39,1.0,1.0,0.0
3,53,0,1.0,234721,1.0,7,40,1,6,0.0,4,0.0,0.0,0.0,0.0,39,0.0,0.0,1.0
4,28,0,1.0,338409,0.0,13,40,9,10,0.0,4,1.0,0.0,0.0,1.0,5,0.0,0.0,1.0


## Train-Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(\
     X, y, test_size=0.33, random_state=42)

## Scaling

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [25]:
MLA=[LogisticRegression(),\
     RandomForestClassifier(),\
     ExtraTreesClassifier(),\
     SVC(),\
     GradientBoostingClassifier(),\
     BaggingClassifier(),\
     CatBoostClassifier()\
     
     
]

In [26]:
MLA

[LogisticRegression(),
 RandomForestClassifier(),
 ExtraTreesClassifier(),
 SVC(),
 GradientBoostingClassifier(),
 BaggingClassifier(),

In [27]:
#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = ShuffleSplit(n_splits = 10, test_size = .20, train_size = .80, \
                                                random_state = 0 )
                    # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Mean Accuracy', 'MLA Test Mean Accuracy', \
                'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = y.copy()

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    print("Running the classification on %s" %(MLA_name))
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = cross_validate(alg, X, y, cv = cv_split,return_train_score=\
                                             True, scoring='accuracy')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Mean Accuracy'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Mean Accuracy'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, 
    #should statistically capture 99.7% of the subsets
    #MLA_compare.loc[row_index, 'MLA Test Error 3*STD'] = cv_results['test_score'].std()*3   
    #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(X, y)
    MLA_predict[MLA_name] = alg.predict(X)
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Mean Accuracy'], ascending = False, inplace = True)
MLA_compare
#MLA_predict

Running the classification on LogisticRegression


c:\users\aman\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\aman\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

Running the classification on RandomForestClassifier
Running the classification on ExtraTreesClassifier
Running the classification on SVC
Running the classification on GradientBoostingClassifier
Running the classification on BaggingClassifier
Running the classification on CatBoostClassifier
Learning rate set to 0.041445
0:	learn: 0.6504911	total: 159ms	remaining: 2m 38s
1:	learn: 0.6123428	total: 171ms	remaining: 1m 25s
2:	learn: 0.5797644	total: 185ms	remaining: 1m 1s
3:	learn: 0.5519474	total: 203ms	remaining: 50.5s
4:	learn: 0.5231790	total: 216ms	remaining: 42.9s
5:	learn: 0.5025129	total: 228ms	remaining: 37.8s
6:	learn: 0.4823867	total: 241ms	remaining: 34.2s
7:	learn: 0.4660990	total: 253ms	remaining: 31.4s
8:	learn: 0.4510574	total: 266ms	remaining: 29.3s
9:	learn: 0.4371054	total: 279ms	remaining: 27.6s
10:	learn: 0.4250289	total: 292ms	remaining: 26.2s
11:	learn: 0.4135372	total: 304ms	remaining: 25s
12:	learn: 0.4040944	total: 316ms	remaining: 24s
13:	learn: 0.3970147	total:

164:	learn: 0.2906848	total: 2.38s	remaining: 12.1s
165:	learn: 0.2905895	total: 2.39s	remaining: 12s
166:	learn: 0.2904969	total: 2.41s	remaining: 12s
167:	learn: 0.2902537	total: 2.42s	remaining: 12s
168:	learn: 0.2901375	total: 2.43s	remaining: 12s
169:	learn: 0.2900386	total: 2.45s	remaining: 12s
170:	learn: 0.2899541	total: 2.46s	remaining: 11.9s
171:	learn: 0.2898711	total: 2.47s	remaining: 11.9s
172:	learn: 0.2897538	total: 2.49s	remaining: 11.9s
173:	learn: 0.2894199	total: 2.5s	remaining: 11.9s
174:	learn: 0.2893392	total: 2.51s	remaining: 11.9s
175:	learn: 0.2893043	total: 2.53s	remaining: 11.8s
176:	learn: 0.2891941	total: 2.54s	remaining: 11.8s
177:	learn: 0.2889423	total: 2.55s	remaining: 11.8s
178:	learn: 0.2888437	total: 2.57s	remaining: 11.8s
179:	learn: 0.2887431	total: 2.58s	remaining: 11.8s
180:	learn: 0.2886862	total: 2.59s	remaining: 11.7s
181:	learn: 0.2886199	total: 2.61s	remaining: 11.7s
182:	learn: 0.2885644	total: 2.62s	remaining: 11.7s
183:	learn: 0.2884513	t

336:	learn: 0.2746813	total: 4.55s	remaining: 8.95s
337:	learn: 0.2746218	total: 4.56s	remaining: 8.94s
338:	learn: 0.2745679	total: 4.58s	remaining: 8.93s
339:	learn: 0.2744770	total: 4.59s	remaining: 8.92s
340:	learn: 0.2743912	total: 4.61s	remaining: 8.9s
341:	learn: 0.2743011	total: 4.62s	remaining: 8.89s
342:	learn: 0.2742335	total: 4.63s	remaining: 8.87s
343:	learn: 0.2741503	total: 4.64s	remaining: 8.86s
344:	learn: 0.2741150	total: 4.66s	remaining: 8.85s
345:	learn: 0.2740520	total: 4.67s	remaining: 8.83s
346:	learn: 0.2740052	total: 4.68s	remaining: 8.81s
347:	learn: 0.2739440	total: 4.7s	remaining: 8.8s
348:	learn: 0.2737409	total: 4.71s	remaining: 8.79s
349:	learn: 0.2736175	total: 4.72s	remaining: 8.77s
350:	learn: 0.2735002	total: 4.74s	remaining: 8.76s
351:	learn: 0.2734369	total: 4.75s	remaining: 8.74s
352:	learn: 0.2733728	total: 4.76s	remaining: 8.72s
353:	learn: 0.2733033	total: 4.77s	remaining: 8.7s
354:	learn: 0.2731963	total: 4.78s	remaining: 8.69s
355:	learn: 0.27

...

462:	learn: 0.2691670	total: 6.56s	remaining: 7.6s
463:	learn: 0.2691212	total: 6.57s	remaining: 7.59s
464:	learn: 0.2690681	total: 6.59s	remaining: 7.58s
465:	learn: 0.2690040	total: 6.6s	remaining: 7.56s
466:	learn: 0.2689334	total: 6.61s	remaining: 7.55s
467:	learn: 0.2688885	total: 6.63s	remaining: 7.54s
468:	learn: 0.2688228	total: 6.64s	remaining: 7.52s
469:	learn: 0.2687775	total: 6.66s	remaining: 7.51s
470:	learn: 0.2687429	total: 6.67s	remaining: 7.49s
471:	learn: 0.2686964	total: 6.68s	remaining: 7.48s
472:	learn: 0.2686242	total: 6.7s	remaining: 7.46s
473:	learn: 0.2685871	total: 6.71s	remaining: 7.45s
474:	learn: 0.2685540	total: 6.73s	remaining: 7.44s
475:	learn: 0.2684717	total: 6.74s	remaining: 7.42s
476:	learn: 0.2684139	total: 6.75s	remaining: 7.41s
477:	learn: 0.2683468	total: 6.77s	remaining: 7.39s
478:	learn: 0.2682921	total: 6.78s	remaining: 7.38s
479:	learn: 0.2682432	total: 6.8s	remaining: 7.36s
480:	learn: 0.2682130	total: 6.81s	remaining: 7.35s
481:	learn: 0.26

622:	learn: 0.2609279	total: 8.75s	remaining: 5.29s
623:	learn: 0.2609086	total: 8.76s	remaining: 5.28s
624:	learn: 0.2608563	total: 8.78s	remaining: 5.27s
625:	learn: 0.2608143	total: 8.79s	remaining: 5.25s
626:	learn: 0.2607500	total: 8.81s	remaining: 5.24s
627:	learn: 0.2606946	total: 8.82s	remaining: 5.23s
628:	learn: 0.2606479	total: 8.84s	remaining: 5.21s
629:	learn: 0.2605983	total: 8.86s	remaining: 5.2s
630:	learn: 0.2605332	total: 8.87s	remaining: 5.19s
631:	learn: 0.2604811	total: 8.89s	remaining: 5.17s
632:	learn: 0.2604424	total: 8.9s	remaining: 5.16s
633:	learn: 0.2604053	total: 8.91s	remaining: 5.15s
634:	learn: 0.2603722	total: 8.93s	remaining: 5.13s
635:	learn: 0.2603369	total: 8.94s	remaining: 5.12s
636:	learn: 0.2602766	total: 8.96s	remaining: 5.11s
637:	learn: 0.2602414	total: 8.97s	remaining: 5.09s
638:	learn: 0.2602173	total: 8.99s	remaining: 5.08s
639:	learn: 0.2601725	total: 9s	remaining: 5.06s
640:	learn: 0.2601222	total: 9.02s	remaining: 5.05s
641:	learn: 0.260

784:	learn: 0.2534197	total: 10.9s	remaining: 3s
785:	learn: 0.2533773	total: 11s	remaining: 2.98s
786:	learn: 0.2533324	total: 11s	remaining: 2.97s
787:	learn: 0.2532970	total: 11s	remaining: 2.96s
788:	learn: 0.2532690	total: 11s	remaining: 2.94s
789:	learn: 0.2532165	total: 11s	remaining: 2.93s
790:	learn: 0.2531732	total: 11s	remaining: 2.91s
791:	learn: 0.2531455	total: 11s	remaining: 2.9s
792:	learn: 0.2531000	total: 11.1s	remaining: 2.88s
793:	learn: 0.2530569	total: 11.1s	remaining: 2.87s
794:	learn: 0.2530370	total: 11.1s	remaining: 2.86s
795:	learn: 0.2529976	total: 11.1s	remaining: 2.84s
796:	learn: 0.2529368	total: 11.1s	remaining: 2.83s
797:	learn: 0.2528999	total: 11.1s	remaining: 2.81s
798:	learn: 0.2528407	total: 11.1s	remaining: 2.8s
799:	learn: 0.2528018	total: 11.1s	remaining: 2.79s
800:	learn: 0.2527525	total: 11.2s	remaining: 2.77s
801:	learn: 0.2527284	total: 11.2s	remaining: 2.76s
802:	learn: 0.2526781	total: 11.2s	remaining: 2.75s
803:	learn: 0.2526288	total: 11

952:	learn: 0.2467323	total: 13.1s	remaining: 647ms
953:	learn: 0.2466925	total: 13.1s	remaining: 633ms
954:	learn: 0.2466434	total: 13.1s	remaining: 620ms
955:	learn: 0.2466086	total: 13.2s	remaining: 606ms
956:	learn: 0.2465548	total: 13.2s	remaining: 592ms
957:	learn: 0.2465225	total: 13.2s	remaining: 578ms
958:	learn: 0.2464946	total: 13.2s	remaining: 564ms
959:	learn: 0.2464332	total: 13.2s	remaining: 551ms
960:	learn: 0.2464074	total: 13.2s	remaining: 537ms
961:	learn: 0.2463870	total: 13.2s	remaining: 523ms
962:	learn: 0.2463415	total: 13.3s	remaining: 509ms
963:	learn: 0.2463081	total: 13.3s	remaining: 496ms
964:	learn: 0.2462868	total: 13.3s	remaining: 482ms
965:	learn: 0.2462539	total: 13.3s	remaining: 468ms
966:	learn: 0.2462198	total: 13.3s	remaining: 454ms
967:	learn: 0.2461927	total: 13.3s	remaining: 441ms
968:	learn: 0.2461694	total: 13.3s	remaining: 427ms
969:	learn: 0.2461457	total: 13.4s	remaining: 413ms
970:	learn: 0.2461118	total: 13.4s	remaining: 399ms
971:	learn: 

,MLA Name,MLA Parameters,MLA Train Mean Accuracy,MLA Test Mean Accuracy,MLA Time
6,CatBoostClassifier,{},0.895838,0.863243,13.5973
4,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.865007,0.858759,7.06932
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999958,0.847612,3.59233
5,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...",0.987174,0.83688,0.733705
2,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.999977,0.836189,3.1092
3,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.795474,0.793628,24.1454
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.778432,0.776969,0.289206


In [33]:
bst_model=MLA[6]
bst_model.fit(X_train,y_train)
pred=bst_model.predict(X_test)
print('Accuracy score of model {} is::{}'.format(bst_model,accuracy_score(y_test,pred)))
pd.DataFrame(confusion_matrix(y_test,pred),\
             columns=['Predicted Negative', 'Predicted Positive'],\
             index=['Actual Negative', 'Actual Positive'])

Learning rate set to 0.038422
0:	learn: 0.6548571	total: 32.3ms	remaining: 32.3s
1:	learn: 0.6203375	total: 41.3ms	remaining: 20.6s
2:	learn: 0.5897352	total: 50.5ms	remaining: 16.8s
3:	learn: 0.5621267	total: 59.6ms	remaining: 14.8s
4:	learn: 0.5364898	total: 68.8ms	remaining: 13.7s
5:	learn: 0.5160776	total: 77.8ms	remaining: 12.9s
6:	learn: 0.4965446	total: 86.5ms	remaining: 12.3s
7:	learn: 0.4811410	total: 96.2ms	remaining: 11.9s
8:	learn: 0.4670757	total: 105ms	remaining: 11.6s
9:	learn: 0.4530256	total: 114ms	remaining: 11.3s
10:	learn: 0.4403024	total: 124ms	remaining: 11.1s
11:	learn: 0.4278030	total: 136ms	remaining: 11.2s
12:	learn: 0.4179470	total: 145ms	remaining: 11s
13:	learn: 0.4103199	total: 154ms	remaining: 10.8s
14:	learn: 0.4022327	total: 163ms	remaining: 10.7s
15:	learn: 0.3953661	total: 173ms	remaining: 10.6s
16:	learn: 0.3890613	total: 182ms	remaining: 10.5s
17:	learn: 0.3824797	total: 191ms	remaining: 10.4s
18:	learn: 0.3791921	total: 204ms	remaining: 10.5s
19:	l

170:	learn: 0.2933800	total: 2s	remaining: 9.71s
171:	learn: 0.2931442	total: 2.01s	remaining: 9.7s
172:	learn: 0.2930631	total: 2.03s	remaining: 9.69s
173:	learn: 0.2929165	total: 2.04s	remaining: 9.67s
174:	learn: 0.2927948	total: 2.05s	remaining: 9.65s
175:	learn: 0.2926709	total: 2.06s	remaining: 9.63s
176:	learn: 0.2925721	total: 2.07s	remaining: 9.61s
177:	learn: 0.2923530	total: 2.08s	remaining: 9.59s
178:	learn: 0.2922442	total: 2.09s	remaining: 9.57s
179:	learn: 0.2921185	total: 2.1s	remaining: 9.55s
180:	learn: 0.2920299	total: 2.11s	remaining: 9.54s
181:	learn: 0.2918695	total: 2.12s	remaining: 9.52s
182:	learn: 0.2917090	total: 2.13s	remaining: 9.5s
183:	learn: 0.2915927	total: 2.14s	remaining: 9.49s
184:	learn: 0.2914227	total: 2.15s	remaining: 9.47s
185:	learn: 0.2912950	total: 2.16s	remaining: 9.45s
186:	learn: 0.2911516	total: 2.17s	remaining: 9.45s
187:	learn: 0.2910796	total: 2.18s	remaining: 9.43s
188:	learn: 0.2910010	total: 2.19s	remaining: 9.42s
189:	learn: 0.2908

333:	learn: 0.2765222	total: 4.02s	remaining: 8.01s
334:	learn: 0.2763630	total: 4.03s	remaining: 8s
335:	learn: 0.2763159	total: 4.04s	remaining: 7.99s
336:	learn: 0.2762359	total: 4.05s	remaining: 7.97s
337:	learn: 0.2760908	total: 4.06s	remaining: 7.96s
338:	learn: 0.2759765	total: 4.07s	remaining: 7.94s
339:	learn: 0.2758877	total: 4.08s	remaining: 7.93s
340:	learn: 0.2758161	total: 4.09s	remaining: 7.91s
341:	learn: 0.2756996	total: 4.1s	remaining: 7.89s
342:	learn: 0.2756435	total: 4.11s	remaining: 7.88s
343:	learn: 0.2755675	total: 4.12s	remaining: 7.86s
344:	learn: 0.2753960	total: 4.13s	remaining: 7.85s
345:	learn: 0.2753361	total: 4.14s	remaining: 7.83s
346:	learn: 0.2752399	total: 4.15s	remaining: 7.82s
347:	learn: 0.2751153	total: 4.16s	remaining: 7.8s
348:	learn: 0.2750637	total: 4.17s	remaining: 7.79s
349:	learn: 0.2750018	total: 4.18s	remaining: 7.77s
350:	learn: 0.2749442	total: 4.2s	remaining: 7.76s
351:	learn: 0.2749101	total: 4.21s	remaining: 7.75s
352:	learn: 0.2748

503:	learn: 0.2634832	total: 6s	remaining: 5.9s
504:	learn: 0.2634246	total: 6.01s	remaining: 5.89s
505:	learn: 0.2633973	total: 6.02s	remaining: 5.88s
506:	learn: 0.2633265	total: 6.03s	remaining: 5.87s
507:	learn: 0.2632858	total: 6.04s	remaining: 5.85s
508:	learn: 0.2632122	total: 6.06s	remaining: 5.84s
509:	learn: 0.2631747	total: 6.07s	remaining: 5.83s
510:	learn: 0.2631003	total: 6.08s	remaining: 5.82s
511:	learn: 0.2630638	total: 6.1s	remaining: 5.82s
512:	learn: 0.2629703	total: 6.12s	remaining: 5.81s
513:	learn: 0.2629153	total: 6.15s	remaining: 5.81s
514:	learn: 0.2628760	total: 6.17s	remaining: 5.81s
515:	learn: 0.2628170	total: 6.19s	remaining: 5.81s
516:	learn: 0.2627562	total: 6.21s	remaining: 5.8s
517:	learn: 0.2626732	total: 6.24s	remaining: 5.8s
518:	learn: 0.2626004	total: 6.27s	remaining: 5.81s
519:	learn: 0.2625041	total: 6.29s	remaining: 5.8s
520:	learn: 0.2624592	total: 6.3s	remaining: 5.79s
521:	learn: 0.2623910	total: 6.31s	remaining: 5.78s
522:	learn: 0.2623415

680:	learn: 0.2539164	total: 7.98s	remaining: 3.74s
681:	learn: 0.2539037	total: 7.99s	remaining: 3.73s
682:	learn: 0.2538509	total: 8.01s	remaining: 3.72s
683:	learn: 0.2538015	total: 8.02s	remaining: 3.7s
684:	learn: 0.2537242	total: 8.03s	remaining: 3.69s
685:	learn: 0.2536878	total: 8.04s	remaining: 3.68s
686:	learn: 0.2536360	total: 8.05s	remaining: 3.67s
687:	learn: 0.2535627	total: 8.06s	remaining: 3.65s
688:	learn: 0.2534926	total: 8.07s	remaining: 3.64s
689:	learn: 0.2534275	total: 8.08s	remaining: 3.63s
690:	learn: 0.2533910	total: 8.09s	remaining: 3.62s
691:	learn: 0.2533652	total: 8.1s	remaining: 3.61s
692:	learn: 0.2533223	total: 8.11s	remaining: 3.59s
693:	learn: 0.2532766	total: 8.13s	remaining: 3.58s
694:	learn: 0.2532567	total: 8.14s	remaining: 3.57s
695:	learn: 0.2531764	total: 8.15s	remaining: 3.56s
696:	learn: 0.2531218	total: 8.16s	remaining: 3.55s
697:	learn: 0.2530591	total: 8.17s	remaining: 3.54s
698:	learn: 0.2530240	total: 8.18s	remaining: 3.52s
699:	learn: 0.

840:	learn: 0.2458736	total: 9.61s	remaining: 1.82s
841:	learn: 0.2458099	total: 9.62s	remaining: 1.8s
842:	learn: 0.2457634	total: 9.63s	remaining: 1.79s
843:	learn: 0.2457439	total: 9.64s	remaining: 1.78s
844:	learn: 0.2457300	total: 9.65s	remaining: 1.77s
845:	learn: 0.2456869	total: 9.66s	remaining: 1.76s
846:	learn: 0.2456194	total: 9.67s	remaining: 1.75s
847:	learn: 0.2455338	total: 9.68s	remaining: 1.74s
848:	learn: 0.2455135	total: 9.69s	remaining: 1.72s
849:	learn: 0.2454480	total: 9.7s	remaining: 1.71s
850:	learn: 0.2454103	total: 9.71s	remaining: 1.7s
851:	learn: 0.2453304	total: 9.72s	remaining: 1.69s
852:	learn: 0.2452361	total: 9.73s	remaining: 1.68s
853:	learn: 0.2452076	total: 9.74s	remaining: 1.67s
854:	learn: 0.2451675	total: 9.75s	remaining: 1.65s
855:	learn: 0.2451431	total: 9.76s	remaining: 1.64s
856:	learn: 0.2451014	total: 9.77s	remaining: 1.63s
857:	learn: 0.2450486	total: 9.78s	remaining: 1.62s
858:	learn: 0.2449897	total: 9.8s	remaining: 1.61s
859:	learn: 0.24

,Predicted Negative,Predicted Positive
Actual Negative,7654,542
Actual Positive,901,1649
